<a href="https://www.kaggle.com/code/kaiyoo88/kpop-idols-w2v?scriptVersionId=204300813" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# pip install konlpy

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import kagglehub
import tqdm
from konlpy.tag import Okt # tokenizer for Korean
from gensim.models import Word2Vec, KeyedVectors
import requests
from io import BytesIO

In [6]:
file_path = kagglehub.dataset_download('kaiyoo88/youtube-comments-for-top-100-ranked-kpop-idols')
file_path

'/kaggle/input/youtube-comments-for-top-100-ranked-kpop-idols'

In [7]:
train_data = pd.read_csv(f'{file_path}/youtube_comments.csv')

In [8]:
train_data.head()

,Video_ID,Artist,Comment
0,J6mQx6c0oIM,블랙핑크 제니,멋지다 한국팬들위해 엠카무대도 나오고 서있기만해도 힙하다
1,J6mQx6c0oIM,블랙핑크 제니,지금은 말할 수 있다. 솔직이 제니 도레미파솔라시 음정이 제 위치에 안꽂치고 조금씩...
2,J6mQx6c0oIM,블랙핑크 제니,김태희 얼굴이 보이는건 나뿐인가..
3,J6mQx6c0oIM,블랙핑크 제니,도대체 머라고 떠드는지... 한국에서 먼 소린지\n로제좀 본받아라
4,J6mQx6c0oIM,블랙핑크 제니,라이브 다 들려 하.. 퀸제니 진짜 미치겠다


In [9]:
print(len(train_data)) # #comments

120101


In [10]:
# NULL check
print(train_data.isnull().values.any())
train_data = train_data.dropna(how = 'any') 
print(train_data.isnull().values.any())
print(len(train_data)) 

False
False
120101


In [11]:
# Only Korean comments were collected during data collection process.
# remove all other charcaters other than Hangeul.
train_data['Comment'] = train_data['Comment'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","", regex=True)

In [13]:
# tokenizer for Korean comments. Use nltk or other tokenizer for English comments
okt = Okt() 

In [14]:
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

In [15]:
tokenized_data = []
for sentence in tqdm.tqdm(train_data['Comment']):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    tokenized_data.append(stopwords_removed_sentence)

100%|██████████| 120101/120101 [05:31<00:00, 362.13it/s]


In [1]:
# Comments length distribution
print('max comment length :',max(len(comment) for comment in tokenized_data))
print('avg comment length :',sum(map(len, tokenized_data))/len(tokenized_data))
plt.hist([len(comment) for comment in tokenized_data], bins=50)
plt.xlabel('length of samples')
plt.ylabel('number of samples')
plt.show()

NameError: name 'tokenized_data' is not defined

## Train Youtube comments on idol groups with Word2Vec 

In [ ]:
#Run following and restart session if error in smart_open circular import
#pip install -U smart_open or
#!pip install smart_open==5.2.1

In [ ]:
# FastText pretrained Korean embedding
url = 'https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ko.300.bin.gz'
response = requests.get(url)
compressed_file = BytesIO(response.content)

In [ ]:
# Load pretraiend Korean embedding
pretrained_model = KeyedVectors.load_word2vec_format(compressed_file, binary=True)
print(f"#Word embedding vectors: {len(word2vec_model.key_to_index)}")

In [ ]:
# Load pretraiend Korean embedding
# pretrained_model = KeyedVectors.load_word2vec_format('한국어 pretrained 모델', binary=True)

# transform to Word2Vec for training
# model = Word2Vec(sentences = tokenized_data, vector_size = 100, window = 5, min_count = 5, workers = 4, sg = 0)
model = Word2Vec(vector_size=pretrained_model.vector_size, window=5, min_count=5, sg=0)
model.build_vocab([list(pretrained_model.key_to_index)], update=True)  # 사전 학습된 단어 추가
model.build_vocab(tokenized_data, update=True)  # 추가 학습할 데이터 추가

# train on my dataset with pretrained embedding weight
model.train(tokenized_data, total_examples=model.corpus_count, epochs=model.epochs)

In [ ]:
model.wv.save_word2vec_format('idol_w2v') # 모델 저장

In [ ]:
model.wv.vectors.shape # 100 dimension

In [ ]:
print(model.wv.most_similar("웬디"))

In [ ]:
print(model.wv.most_similar("조이"))

## Visualisation for Word Embedding

In [ ]:
# Generate 2 tsv files for embedding visualisation.
!python -m gensim.scripts.word2vec2tensor --input idol_w2v --output idol_w2v

In [ ]:
# https://projector.tensorflow.org/
# Load ko_w2v_tensor.tsv and ko_w2v_metadata.tsv 